In [ ]:
import json

In [ ]:
[x.delete() for x in Manuscript.objects.filter(lib_code=None)]

In [ ]:
items = MsImage.objects.filter(manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    ms = Manuscript.objects.get(legacy_pk=ms_id)
    x.manuscript = ms
    x.save()
        

In [ ]:
items = MsPart.objects.filter(part_of_manuscript=None)
items.count()

In [ ]:
for x in items:
    try:
        ms_id = int(json.loads(x.orig_data_csv)["idmanuscripts"])
    except:
        x.delete()
        continue
    try:
        ms = Manuscript.objects.get(legacy_pk=ms_id)
    except:
        print(ms_id, x.id)
        continue
    x.part_of_manuscript = ms
    x.save()

In [ ]:
# link lib to lit

In [ ]:
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine
from django.conf import settings
from django.core.exceptions import ObjectDoesNotExist

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [ ]:
source_name = "litkat"
query = f"SELECT * FROM {source_name}"
df = pd.read_sql(query, con=db_connection)

In [ ]:
df

In [ ]:
for lib_code, ndf in df.groupby("libCode"):
    lib = Bibliothek.objects.filter(lib_code=lib_code).first()
    lit_ids = set(ndf["lit_id"].to_list())
    items = Literatur.objects.filter(legacy_pk__in=lit_ids)
    lib.literatur.add(*items)    

In [ ]:
lit_ids